# **Quick introduction**

As we know, cereals have been present in our lives since the beginning of the agrarian era. Over the centuries, we have developed agriculture, which has brought us into more modern times. At the moment the prices of cereals fluctuate year by year and the stock exchange reacts to many situations in the world by modifying the prices. In the following short analysis you can see how grain prices have developed over the last 30 years.

The analysis includes
* Average price of three cereal types (corn, rice, wheat) over 30 year - excluding year 2022 
* Percentage change in prices year on year for the last 29 years.
* Visualisations of the above parameters.
* The programming language used for analysis and visualisation: R.

Enjoy!




# **Let's load and look at the data**

In [1]:
library(tidyverse)
library(lubridate)

In [38]:
rice_wheat_corn_prices <- read_csv('../input/-cerial-prices-changes-within-last-30-years/rice_wheat_corn_prices.csv')

Let's see the data frame

In [ ]:
rice_wheat_corn_prices

Before we start the analysis, let's look at the data structure and change a few things to get to the next steps.

In [5]:
glimpse(rice_wheat_corn_prices)

Using the match() function I change the values in the column from "Jan" to "1" etc. This may be helpful for further analysis. R sometimes does not recognise abbreviations.

In [6]:
rice_wheat_corn_prices$Month <-  match(rice_wheat_corn_prices$Month, month.abb)
glimpse(rice_wheat_corn_prices)

Let's look for the missing values

In [39]:
  # any missing data?

sum(is.na(rice_wheat_corn_prices))

As you can see we have 5 NA's. Let's do a quick summary of the available variables to find out which columns contain the missing values.

In [8]:
# check the numbers
summary(rice_wheat_corn_prices)

Scrolling up to the data table, we see that our NA's are in the row for 2022. Luckily, we didn't want to include this date in the analysis so the missing values will be automatically filtered out.

# **Annual average price for each cereal type**

In [9]:
yearly_price <- rice_wheat_corn_prices %>%
  filter(Year != 2022) %>%
  group_by(Year) %>% 
  summarise_at(vars('Price_wheat_ton', 'Price_rice_ton', 'Price_corn_ton'), mean)

yearly_price

I modify the columns using pivot_longer() so that I can visualise the results more easily.

In [10]:
yearly_price_pivoted <- yearly_price %>% 
  pivot_longer(2:4, names_to = 'Cerial_type', values_to = "Avg_Price")

Change settings to generate larger graphs.

In [26]:
options(repr.plot.width=18, repr.plot.height=11)


Price visualisation over the years.

In [37]:
# Viz it
ggplot(yearly_price_pivoted, aes(Year, Avg_Price, group = Cerial_type)) +
  geom_line(aes(color = Cerial_type)) +
  geom_point(aes(color = Cerial_type))+
  scale_y_continuous(breaks = seq(0, 800, by = 25))+
  scale_x_continuous(breaks = seq(1992, 2021, by = 1)) +
  geom_vline(xintercept = 2008 , linetype="dashed", color = "grey60") +
  annotate(geom ="text",x = 2010, y = 355, label="Global Financial \n Crisis 2007-2008", size = 5, color = "grey60") +

  labs(x = "Year", y = "Average price per year", title = "Average price of three cereal types over 30 years")+
  theme_bw() +
  theme(legend.position = "top",
        plot.title = element_text(hjust = 0.5, size = 16),
        axis.title.x = element_text(margin = margin(t = 10), size = 15,),
        axis.title.y = element_text(margin = margin(r = 10), size = 15),
        axis.text.x = element_text(angle = 45, vjust = 1, hjust = 1, size = 12),
        axis.text = element_text(size = 12)) +
  scale_color_manual(name = "Cereal types", labels = c("Corn", "Rice", "Wheat"), values = c("gold2", "gray30", "darkolivegreen"))

# **Year-on-year percentage change**

In [13]:
  # get the year on year percentage change
yearly_price_percentage_change <-  yearly_price_pivoted %>% 
  group_by(Cerial_type) %>% 
  mutate(
    Year_on_year = (Avg_Price - lag(Avg_Price)) / lag(Avg_Price)
  )
  # round the numbers
yearly_price_percentage_change <- yearly_price_percentage_change %>% 
  mutate(
    Year_on_year = round(Year_on_year * 100, 2)
  )

yearly_price_percentage_change

Vizualize it.

In [ ]:
  ggplot(yearly_price_percentage_change, aes(Year, Year_on_year, fill = Cerial_type)) +
    geom_col(position = "dodge") +
    scale_x_continuous(breaks = seq(1992, 2021, by = 1))+
    scale_y_continuous(breaks = seq(-40, 100, by = 10)) + 
    facet_grid(rows = vars(Cerial_type)) +
    geom_hline(yintercept = 0 , linetype="dashed", color = "gray80") +
    theme_bw() +
    labs(x = "Year", y = "Percentage change", title = "Percentage change in prices year on year for the last 29 years") +
    theme(legend.position = "top",
          plot.title = element_text(hjust = 0.5, size = 16),
          axis.title.x = element_text(margin = margin(t = 10), size = 15,),
          axis.title.y = element_text(margin = margin(r = 10), size = 15),
          axis.text.x = element_text(angle = 45, vjust = 1, hjust = 1, size = 12),
          axis.text = element_text(size = 12),
          strip.text.y = element_blank()) +
    scale_fill_manual(name = "Cereal types", labels=c("Corn","Rice","Wheat"), values = c("gold2", "grey", "darkolivegreen"))


**Conclusions:**
The price of cereals is not stable and is strongly influenced by world releases. This is clearly visible in the graph showing the price changes over the years. The year 2008 and the crisis at that time shot up the price of all three types of cereals